In [139]:
import pandas as pd
import numpy as np

In [141]:
file_path = 'Data.csv'
data = pd.read_csv(file_path, sep=';', index_col=0)

data.head()

,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,31.01.2020,...,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,237056.39,...,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,419598.74,...,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,503465.03,...,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,694348.63,...,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,939424.54,...,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05


In [143]:
# Convert all data to numeric, ignoring errors for non-convertible values
df = data.apply(pd.to_numeric, errors='coerce')

# Extract the 'Origination Amount' column
origination_amounts = df['Origination Amount']

# Compute the repayment percentages by dividing each repayment by the origination amount
repayment_percentages = df.drop(columns=['Origination Amount']).div(origination_amounts, axis=0) * 100

# Round the percentages to two decimal places for readability
repayment_percentages = repayment_percentages.round(2)

# Display the repayment percentages
repayment_percentages.head()

,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,31.01.2020,29.02.2020,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
31.05.2019,14.4,33.26,13.26,9.26,7.35,5.38,4.27,3.24,2.37,1.68,1.16,0.93,0.63,0.53,0.37,0.30,0.24,0.18,0.17,0.11
30.06.2019,0.0,12.81,27.71,11.39,8.93,8.21,6.15,4.65,3.86,3.03,2.35,1.83,1.49,1.27,0.85,0.73,0.58,0.48,0.40,0.34
31.07.2019,0.0,0.00,14.33,27.51,11.26,8.19,6.62,6.13,4.69,3.94,2.82,2.41,1.79,1.58,1.19,1.03,0.84,0.60,0.57,0.47
31.08.2019,0.0,0.00,0.00,12.88,32.50,11.05,9.93,7.06,5.53,4.55,3.32,2.68,2.02,1.59,1.21,0.88,0.72,0.56,0.42,0.37
30.09.2019,0.0,0.00,0.00,0.00,13.73,27.10,9.61,8.47,6.48,5.54,4.33,4.07,3.15,2.23,1.99,1.65,1.33,1.18,0.98,0.81


In [145]:
# Initialize a DataFrame to store expected repayment percentages for months 1 to 30
expected_p = pd.DataFrame(index=df.index, columns=range(1, 31))

# Function to calculate months between two dates
def months_between(start_date, end_date):
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1

# Fill in the observed p_i for i = 1 and i = 2 (if available)
for vintage in df.index:
    # Get the origination amount
    orig_amount = origination_amounts[vintage]
    
    # Get the repayment dates (columns)
    repayment_dates = df.columns[1:]
    
    # Convert origination date to datetime
    orig_date = pd.to_datetime(vintage, format='%d.%m.%Y')
    
    # Convert repayment dates to datetime
    repayment_dates_dt = pd.to_datetime(repayment_dates, format='%d.%m.%Y')
    
    # Calculate the months since origination for each repayment date
    months = [months_between(orig_date, date) for date in repayment_dates_dt]
    
    # Create a Series of observed repayments with the corresponding month numbers
    repayments = df.loc[vintage, repayment_dates].values
    months = np.array(months)
    
    # Map observed repayments to expected_p DataFrame
    for month, repayment in zip(months, repayments):
        if 1 <= month <= 30:
            expected_p.at[vintage, month] = repayment / orig_amount
    
    # Handle December 2020 vintage special case for p_2
    if vintage == '31.12.2020':
        # If p_1 is available
        if expected_p.at[vintage, 1] is not None and not np.isnan(expected_p.at[vintage, 1]):
            expected_p.at[vintage, 2] = 2 * expected_p.at[vintage, 1]
        else:
            # If p_1 is not available, set p_2 to zero
            expected_p.at[vintage, 2] = 0

# For vintages where p_2 is not observed, fill p_2 if possible
for vintage in expected_p.index:
    if pd.isnull(expected_p.at[vintage, 2]) or expected_p.at[vintage, 2] == 0:
        if not pd.isnull(expected_p.at[vintage, 1]) and expected_p.at[vintage, 1] != 0:
            expected_p.at[vintage, 2] = expected_p.at[vintage, 1]  # Assuming p_2 = p_1 if p_2 is missing
        else:
            expected_p.at[vintage, 2] = 0

# Now compute p_i for i from 3 to 30 using the adjusted formula
for vintage in expected_p.index:
    p2 = expected_p.at[vintage, 2]
    if pd.isnull(p2) or p2 == 0:
        continue  # Cannot compute future p_i without p2
    for i in range(3, 31):
        # Sum of p_j from j=1 to i-1
        sum_pj = sum([expected_p.at[vintage, j] if expected_p.at[vintage, j] is not None else 0 for j in range(1, i)])
        # Compute the inner value for ln()
        inner_ln = 1 + (1 - sum_pj)
        if inner_ln <= 0:
            p_i = 0
        else:
            # Adjusted formula: Multiply by (1 - (i - 1)/30) outside the log
            p_i = p2 * np.log(inner_ln) * (1 - (i - 1) / 30)
            p_i = max(p_i, 0)  # Ensure p_i is not negative
        expected_p.at[vintage, i] = p_i

# Replace any remaining NaN with zeros
expected_p = expected_p.fillna(0)

# Multiply by 100 to express percentages
expected_p_percent = expected_p * 100

# Round the percentages to two decimal places for readability
expected_p_percent = expected_p_percent.round(2)

# Display the expected repayment percentages
print("Expected Repayment Percentages (% of Origination Amount):")
print(expected_p_percent)

Expected Repayment Percentages (% of Origination Amount):
               1      2      3      4     5     6     7     8     9     10  \
31.05.2019  14.40  33.26  13.07   9.92  7.42  5.50  4.05  2.98  2.19  1.62   
30.06.2019  12.81  27.71  12.07   9.68  7.69  6.07  4.76  3.73  2.92  2.29   
31.07.2019  14.33  27.51  11.77   9.44  7.50  5.92  4.65  3.65  2.86  2.25   
31.08.2019  12.88  32.50  13.22  10.13  7.66  5.74  4.27  3.17  2.36  1.76   
30.09.2019  13.73  27.10  11.75   9.46  7.56  5.99  4.73  3.73  2.94  2.32   
31.10.2019  14.63  29.91  12.32   9.66  7.49  5.76  4.41  3.37  2.58  1.97   
30.11.2019  14.31  30.70  12.55   9.77  7.52  5.74  4.36  3.30  2.50  1.90   
31.12.2019  14.13  29.10  12.21   9.65  7.55  5.86  4.53  3.50  2.70  2.08   
31.01.2020  14.90  27.97  11.80   9.41  7.44  5.84  4.57  3.56  2.78  2.17   
29.02.2020  14.75  31.97  12.74   9.79  7.43  5.58  4.18  3.12  2.33  1.75   
31.03.2020  13.15  28.80  12.30   9.76  7.67  5.98  4.64  3.60  2.78  2.16   
30.04.

In [147]:
# Now compute the expected cash flows (in monetary terms) for each vintage
expected_cash_flows = expected_p.mul(origination_amounts, axis=0)

# Display the expected cash flows for each vintage
print("\nExpected Cash Flows:")
print(expected_cash_flows.round(2))


Expected Cash Flows:
                    1           2           3           4           5   \
31.05.2019  1443069.08  3332200.33  1309092.27   993400.07   743315.33   
30.06.2019  1392751.60  3011884.91  1311948.01  1051726.38   835522.91   
31.07.2019  1537650.24  2953335.55  1263671.93  1012942.25   804867.04   
31.08.2019  1617681.94  4082016.00  1660240.13  1272583.80   962376.76   
30.09.2019  1992242.84  3930445.60  1705046.80  1372759.82  1095889.64   
31.10.2019  2289453.76  4682354.31  1928213.98  1511452.70  1172055.65   
30.11.2019  2162283.09  4637701.69  1896716.10  1476395.09  1136031.25   
31.12.2019  2402403.37  4947764.21  2076428.43  1641186.11  1284120.88   
31.01.2020  2502066.86  4696910.48  1981174.53  1580521.64  1249476.93   
29.02.2020  2833811.35  6142911.08  2449008.85  1881678.50  1427522.98   
31.03.2020  2843285.54  6228477.78  2661170.40  2111822.50  1659297.41   
30.04.2020  3332800.29  6476251.76  2691065.70  2126696.87  1664040.48   
31.05.2020  3575

In [149]:
# Compute the discount factors for each period (monthly discounting)
annual_discount_rate = 0.025  # 2.5%
monthly_discount_rate = (1 + annual_discount_rate) ** (1/12) - 1

# Create a discount factor array for 30 periods
discount_factors = [(1 / ((1 + monthly_discount_rate) ** i)) for i in range(1, 31)]

# Convert discount_factors to a DataFrame to align with expected_cash_flows
discount_factors_df = pd.DataFrame([discount_factors] * len(expected_cash_flows), index=expected_cash_flows.index)

# Apply discount factors to expected cash flows
discounted_cash_flows = expected_cash_flows * discount_factors_df.values

# Sum the discounted cash flows for each vintage to get the present value
present_values = discounted_cash_flows.sum(axis=1)

# Display the present value of expected cash flows for each vintage
print("\nPresent Value of Expected Cash Flows:")
print(present_values.round(2))


Present Value of Expected Cash Flows:
31.05.2019     9892397.59
30.06.2019    10618483.89
31.07.2019    10488320.84
31.08.2019    12384637.17
30.09.2019    14152242.75
31.10.2019    15381281.15
30.11.2019    14864663.55
31.12.2019    16679175.74
31.01.2020    16434008.62
29.02.2020    18946418.99
31.03.2020    21192064.73
30.04.2020    21908741.42
31.05.2020    22137522.84
30.06.2020    24430948.31
31.07.2020    25095686.61
31.08.2020    25519705.08
30.09.2020    25290912.91
31.10.2020    27284113.57
30.11.2020    29236256.14
31.12.2020    29875174.45
dtype: float64


In [151]:
# Total estimated portfolio value
our_estimate = present_values.sum()

# Client's estimate of the portfolio value
clients_estimate = 84993122.67  # CHF

# Compute the absolute difference
absolute_difference = abs(our_estimate - clients_estimate)

# Compute the relative difference (as a percentage)
relative_difference = (absolute_difference / clients_estimate) * 100

# Determine if the difference falls below Jakob's threshold
threshold = 500000  # CHF

difference_within_threshold = absolute_difference < threshold

# Print the results
print(f"Our estimated portfolio value: CHF {our_estimate:,.2f}")
print(f"Client's estimated portfolio value: CHF {clients_estimate:,.2f}")
print(f"Absolute difference: CHF {absolute_difference:,.2f}")
print(f"Relative difference: {relative_difference:.2f}%")

if difference_within_threshold:
    print("The difference is less than CHF 500,000 and is acceptable.")
else:
    print("The difference exceeds CHF 500,000 and is not acceptable.")

Our estimated portfolio value: CHF 391,812,756.35
Client's estimated portfolio value: CHF 84,993,122.67
Absolute difference: CHF 306,819,633.68
Relative difference: 360.99%
The difference exceeds CHF 500,000 and is not acceptable.
